In [ ]:
!nvidia-smi

Sat Jul 11 21:24:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    25W /  75W |    271MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, LSTM, Embedding, Bidirectional, GlobalAveragePooling1D, GRU, Conv1D
import tensorflow_hub as hub
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Carga de datos

In [ ]:
train_labels = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Redes Neuronales/data_taller_2/train_labels.csv')

l_text = []

for i in range(train_labels.shape[0]):
    ID = train_labels['id'][i] 
    train_source = open('/content/drive/My Drive/Colab Notebooks/Redes Neuronales/data_taller_2/train_source_tweets.txt','r')
    for linea in train_source: 
        if str(ID) == linea.split()[0]:
            t = linea.split()[1:]
            text = ''
            for p in t:
                text += p+' '
            l_text.append(text)
            break
    train_source.close()
    
train_labels['text'] = l_text
train_labels

,label,id,text
0,non-rumor,692735698349199360,north korea 'planning some kind of rocket laun...
1,unverified,525008463819464704,"meet kevin vickers, the hero who shot down the..."
2,false,505611045897924608,"15 year old who ""swatted"" gamer convicted of d..."
3,non-rumor,693466724822323200,audio recordings reveal cpr started 11 minutes...
4,true,510922415468449792,awful mt @scclemons:uk aid worker david haines...
...,...,...,...
1921,true,515598200796090369,#workplace hero shoots #altonnolen #stabbings ...
1922,unverified,535257207991205888,russian boy shocked by electricity now claims ...
1923,true,524944399890124801,recap: gunman shot dead inside parliament buil...
1924,false,519965814175567872,a vandal painted a brilliantly simple penis on...


# Tokenizer

In [ ]:
import re
import string

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    #text = re.sub('\w*\d\w*', '', text)
    return text

train_labels['clean_text'] = train_labels['text'].apply(lambda x:clean_text(x))
train_labels

,label,id,text,clean_text
0,non-rumor,692735698349199360,north korea 'planning some kind of rocket laun...,north korea planning some kind of rocket launc...
1,unverified,525008463819464704,"meet kevin vickers, the hero who shot down the...",meet kevin vickers the hero who shot down the ...
2,false,505611045897924608,"15 year old who ""swatted"" gamer convicted of d...",15 year old who swatted gamer convicted of dom...
3,non-rumor,693466724822323200,audio recordings reveal cpr started 11 minutes...,audio recordings reveal cpr started 11 minutes...
4,true,510922415468449792,awful mt @scclemons:uk aid worker david haines...,awful mt scclemonsuk aid worker david haines b...
...,...,...,...,...
1921,true,515598200796090369,#workplace hero shoots #altonnolen #stabbings ...,workplace hero shoots altonnolen stabbings beh...
1922,unverified,535257207991205888,russian boy shocked by electricity now claims ...,russian boy shocked by electricity now claims ...
1923,true,524944399890124801,recap: gunman shot dead inside parliament buil...,recap gunman shot dead inside parliament build...
1924,false,519965814175567872,a vandal painted a brilliantly simple penis on...,a vandal painted a brilliantly simple penis on...


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 1000
embedding_dim = 256
max_length = 256
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_labels["clean_text"])
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_labels["clean_text"])
train_padded = pad_sequences(train_sequences, padding = padding_type, maxlen = max_length)
train_padded.shape

(1926, 256)

# Encoder

In [ ]:
sent_to_id = {"false":0, "non-rumor":1, "true":2, "unverified":3}
train_labels["label_id"] = train_labels['label'].map(sent_to_id)
train_labels

,label,id,text,clean_text,label_id
0,non-rumor,692735698349199360,north korea 'planning some kind of rocket laun...,north korea planning some kind of rocket launc...,1
1,unverified,525008463819464704,"meet kevin vickers, the hero who shot down the...",meet kevin vickers the hero who shot down the ...,3
2,false,505611045897924608,"15 year old who ""swatted"" gamer convicted of d...",15 year old who swatted gamer convicted of dom...,0
3,non-rumor,693466724822323200,audio recordings reveal cpr started 11 minutes...,audio recordings reveal cpr started 11 minutes...,1
4,true,510922415468449792,awful mt @scclemons:uk aid worker david haines...,awful mt scclemonsuk aid worker david haines b...,2
...,...,...,...,...,...
1921,true,515598200796090369,#workplace hero shoots #altonnolen #stabbings ...,workplace hero shoots altonnolen stabbings beh...,2
1922,unverified,535257207991205888,russian boy shocked by electricity now claims ...,russian boy shocked by electricity now claims ...,3
1923,true,524944399890124801,recap: gunman shot dead inside parliament buil...,recap gunman shot dead inside parliament build...,2
1924,false,519965814175567872,a vandal painted a brilliantly simple penis on...,a vandal painted a brilliantly simple penis on...,0


In [ ]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(train_labels.label_id)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
Y = onehot_encoder.fit_transform(integer_encoded)
Y

array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.]])

# Modelo



In [ ]:
def model_simple():
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    return model

def model_s():
    model = Sequential()
    model.add(Dense(256, input_shape=[max_length], activation='relu'))
    model.add(Dense(4,activation='softmax'))
    return model

def model_lstm():
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
    model.add(LSTM(50, dropout=0.5))
    model.add(Dense(4, activation='softmax'))
    return model

model = model_simple()
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 256, 256)          256000    
_________________________________________________________________
global_average_pooling1d_3 ( (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 260       
Total params: 272,708
Trainable params: 272,708
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Entrenamiento

In [ ]:
X = train_padded
# Entrenamiento
history = model.fit(X, Y, epochs=70, batch_size=32, validation_split=0.25, verbose=1,callbacks=[])

Epoch 1/70
46/46 [==============================] - 1s 12ms/step - loss: 1.3802 - accuracy: 0.2749 - val_loss: 1.3789 - val_accuracy: 0.2614
Epoch 2/70
46/46 [==============================] - 0s 10ms/step - loss: 1.3727 - accuracy: 0.2770 - val_loss: 1.3775 - val_accuracy: 0.2614
Epoch 3/70
46/46 [==============================] - 0s 9ms/step - loss: 1.3707 - accuracy: 0.3082 - val_loss: 1.3731 - val_accuracy: 0.3465
Epoch 4/70
46/46 [==============================] - 0s 9ms/step - loss: 1.3644 - accuracy: 0.4356 - val_loss: 1.3686 - val_accuracy: 0.3859
Epoch 5/70
46/46 [==============================] - 0s 9ms/step - loss: 1.3561 - accuracy: 0.4024 - val_loss: 1.3578 - val_accuracy: 0.4793
Epoch 6/70
46/46 [==============================] - 0s 9ms/step - loss: 1.3405 - accuracy: 0.4259 - val_loss: 1.3428 - val_accuracy: 0.3651
Epoch 7/70
46/46 [==============================] - 0s 10ms/step - loss: 1.3141 - accuracy: 0.4806 - val_loss: 1.3148 - val_accuracy: 0.4813
Epoch 8/70
46/46 

# Predictions

In [ ]:
l_id = []
l_text = []

test_source = open('/content/drive/My Drive/Colab Notebooks/Redes Neuronales/data_taller_2/test_source_tweets.txt','r')
for linea in test_source: 
    ID = linea.split()[0]
    t = linea.split()[1:]
    text = ''
    for p in t:
        text += p+' '
    l_id.append(ID)
    l_text.append(text)

test_source.close()

sample = pd.DataFrame()
sample['id'] = l_id
sample['text'] = l_text
sample['clean_text'] = sample['text'].apply(lambda x:clean_text(x))
sample

,id,text,clean_text
0,689719109785907200,death toll from #bachakhanuniattack rises to a...,death toll from bachakhanuniattack rises to at...
1,531300385248800768,"the head of isis, abu bakr al-baghdadi, is sai...",the head of isis abu bakr albaghdadi is said t...
2,652992524504600576,steve jobs was adopted. his biological father ...,steve jobs was adopted his biological father w...
3,509473920060104704,microsoft to buy mojang - report URL $2 billio...,microsoft to buy mojang report url 2 billion ...
4,651321040119963648,el chapo puts $100 million bounty on donald tr...,el chapo puts 100 million bounty on donald tru...
...,...,...,...
208,642432477185867776,the stickers on fruit are edible.,the stickers on fruit are edible
209,693236688538243074,"is the truth out there? in nod to @thexfiles, ...",is the truth out there in nod to thexfiles cia...
210,504131150429061121,breaking: fbi uncovers new audio from neighbor...,breaking fbi uncovers new audio from neighbor ...
211,692004901455556608,.@potus just announced new reforms to address ...,potus just announced new reforms to address th...


In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(sample["clean_text"])
word_index = tokenizer.word_index

test_sequences = tokenizer.texts_to_sequences(sample["clean_text"])
test_padded = pad_sequences(test_sequences, padding = padding_type, maxlen = max_length)
test_padded.shape

(213, 256)

In [ ]:
Y_pred = np.argmax(model.predict(test_padded), axis=-1)
sample['label_id_pred'] = Y_pred
sent_to_label = {0:"false", 1:"non-rumor", 2:"true", 3:"unverified"}
sample["label_pred"] = sample['label_id_pred'].map(sent_to_label)
sample_sub = pd.DataFrame({'label':sample['label_pred'], 'id':sample['id']})
sample_sub.to_csv('sample_submission_mateneuronas.csv', index=False)
sample_sub.head()

,label,id
0,false,689719109785907200
1,false,531300385248800768
2,false,652992524504600576
3,true,509473920060104704
4,false,651321040119963648
